In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import os
import time
from datetime import datetime, timedelta
import random
import shutil

# Initialisation de la SparkSession – point d’entrée pour toutes les fonctionnalités Spark
spark = SparkSession.builder \
    .appName("PySpark ETL Example") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "1g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

# Réduction de la verbosité des logs
spark.sparkContext.setLogLevel("WARN")

print("SparkSession créée avec succès")
print(f"Version de Spark : {spark.version}")
print(f"URL Spark UI : {spark.sparkContext.uiWebUrl}")


SparkSession créée avec succès
Version de Spark : 3.3.2
URL Spark UI : http://10.172.249.176:40001


In [0]:
# Définition du schéma pour les ventes
sales_schema = StructType([
    StructField("transaction_id", StringType(), False),
    StructField("customer_id", StringType(), True),
    StructField("product_id", StringType(), False),
    StructField("store_id", StringType(), False),
    StructField("quantity", IntegerType(), False),
    StructField("price", DoubleType(), False),
    StructField("transaction_date", TimestampType(), False),
    StructField("payment_method", StringType(), True)
])


In [0]:
# Génération de données aléatoires simulées
num_records = 100000
transaction_ids = [f"T{i:08d}" for i in range(1, num_records + 1)]
customer_ids = [f"C{random.randint(1, 5000):05d}" for _ in range(num_records)]
product_ids = [f"P{random.randint(1, 500):05d}" for _ in range(num_records)]
store_ids = [f"S{random.randint(1, 50):03d}" for _ in range(num_records)]
quantities = [random.randint(1, 10) for _ in range(num_records)]
prices = [round(random.uniform(5.0, 500.0), 2) for _ in range(num_records)]


In [0]:
# Dates de transaction sur les 30 derniers jours
now = datetime.now()
dates = [(now - timedelta(days=random.randint(0, 30),
                          hours=random.randint(0, 23),
                          minutes=random.randint(0, 59),
                          seconds=random.randint(0, 59))) for _ in range(num_records)]

payment_methods = [random.choice(["Credit Card", "Debit Card", "Cash", "Mobile Payment", None]) for _ in range(num_records)]


In [0]:
# Création de la liste de lignes
data = list(zip(transaction_ids, customer_ids, product_ids, store_ids, quantities, prices, dates, payment_methods))

# Création de la DataFrame Spark
sales_df = spark.createDataFrame(data, schema=sales_schema)

print(f"DataFrame des ventes créé avec {sales_df.count()} lignes")
print("Schéma :")
sales_df.printSchema()

print("Exemple de données :")
sales_df.show(5, truncate=False)

DataFrame des ventes créé avec 100000 lignes
Schéma :
root
 |-- transaction_id: string (nullable = false)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = false)
 |-- store_id: string (nullable = false)
 |-- quantity: integer (nullable = false)
 |-- price: double (nullable = false)
 |-- transaction_date: timestamp (nullable = false)
 |-- payment_method: string (nullable = true)

Exemple de données :
+--------------+-----------+----------+--------+--------+------+-------------------------+--------------+
|transaction_id|customer_id|product_id|store_id|quantity|price |transaction_date         |payment_method|
+--------------+-----------+----------+--------+--------+------+-------------------------+--------------+
|T00000001     |C03534     |P00125    |S025    |2       |366.51|2025-05-06 10:41:36.10273|Credit Card   |
|T00000002     |C01994     |P00247    |S043    |1       |391.22|2025-05-06 08:17:44.10273|Mobile Payment|
|T00000003     |C02254     |P00247   

In [0]:
# Enregistrement de la DataFrame comme table temporaire pour exécution SQL
sales_df.createOrReplaceTempView("sales")

# 1. Analyse via l'API DataFrame
print("\n--- API DataFrame ---")
store_sales = sales_df.groupBy("store_id") \
    .agg(
        F.sum("quantity").alias("total_quantity"),
        F.sum(F.col("quantity") * F.col("price")).alias("total_sales"),
        F.count("transaction_id").alias("transaction_count")
    ) \
    .orderBy(F.desc("total_sales"))

print("Top 5 des magasins par chiffre d'affaires :")
store_sales.show(5)


--- API DataFrame ---
Top 5 des magasins par chiffre d'affaires :
+--------+--------------+------------------+-----------------+
|store_id|total_quantity|       total_sales|transaction_count|
+--------+--------------+------------------+-----------------+
|    S032|         11628|2962817.0700000003|             2115|
|    S041|         11536|2956428.9699999993|             2088|
|    S011|         11265| 2883221.579999999|             2057|
|    S050|         11225|2880593.2000000007|             2028|
|    S028|         11208|         2875564.4|             2030|
+--------+--------------+------------------+-----------------+
only showing top 5 rows



In [0]:
# 2. Analyse via SQL
print("\n--- SQL ---")
top_products_sql = """
SELECT 
    product_id,
    SUM(quantity) as total_quantity,
    SUM(quantity * price) as total_sales,
    COUNT(DISTINCT transaction_id) as transaction_count
FROM 
    sales
GROUP BY 
    product_id
ORDER BY 
    total_sales DESC
LIMIT 5
"""
top_products = spark.sql(top_products_sql)
print("Top 5 des produits par chiffre d'affaires :")
top_products.show()



--- SQL ---
Top 5 des produits par chiffre d'affaires :
+----------+--------------+------------------+-----------------+
|product_id|total_quantity|       total_sales|transaction_count|
+----------+--------------+------------------+-----------------+
|    P00485|          1399| 367920.8800000001|              238|
|    P00050|          1372|361906.63000000006|              238|
|    P00464|          1353| 350878.6999999998|              231|
|    P00299|          1259|344090.49000000005|              223|
|    P00395|          1247| 340597.8699999999|              217|
+----------+--------------+------------------+-----------------+



In [0]:
# 3. Lecture / Écriture de fichiers

# Parquet
output_dir = "sales_data_parquet"
print(f"\nÉcriture en Parquet dans : {output_dir}")
sales_df.write.mode("overwrite").parquet(output_dir)



Écriture en Parquet dans : sales_data_parquet


In [0]:
# 4. Modification dynamique de configuration Spark
print("\n--- Configuration dynamique ---")
current_shuffle_partitions = spark.conf.get("spark.sql.shuffle.partitions")
print(f"Partitions de shuffle actuelles : {current_shuffle_partitions}")

spark.conf.set("spark.sql.shuffle.partitions", "8")
new_shuffle_partitions = spark.conf.get("spark.sql.shuffle.partitions")
print(f"Nouvelles partitions de shuffle : {new_shuffle_partitions}")



--- Configuration dynamique ---
Partitions de shuffle actuelles : 4
Nouvelles partitions de shuffle : 8


In [0]:
# 5. Utilisation du catalogue Spark
print("\n--- Catalogue Spark ---")
print("Tables temporaires :")
for table in spark.catalog.listTables():
    print(f"  {table.name} (Temporaire : {table.isTemporary})")


--- Catalogue Spark ---
Tables temporaires :
  sales_data (Temporaire : False)
  sales (Temporaire : True)


In [0]:
# Création d'une base et d'une table Hive persistante (si activé)
try:
    spark.sql("CREATE DATABASE IF NOT EXISTS sales_db")
    spark.sql("USE sales_db")
    sales_df.write.mode("overwrite").saveAsTable("sales_history")

    print("\nBases de données disponibles :")
    for db in spark.catalog.listDatabases():
        print(f"  {db.name}")

    print("\nTables dans sales_db :")
    for table in spark.catalog.listTables("sales_db"):
        print(f"  {table.name} (Temporaire : {table.isTemporary})")

    print("\nRequête sur la table persistante :")
    spark.sql("SELECT COUNT(*) as row_count FROM sales_db.sales_history").show()

except Exception as e:
    print(f"⚠️ Impossible de créer une table persistante : {e}")


Bases de données disponibles :
  default
  sales_db
  ventes_db

Tables dans sales_db :
  sales_history (Temporaire : False)
  sales (Temporaire : True)

Requête sur la table persistante :
+---------+
|row_count|
+---------+
|   100000|
+---------+



In [0]:
# 6. Arrêt propre de Spark
print("\nArrêt de SparkSession")
spark.stop()
print("SparkSession arrêtée")

# Nettoyage des fichiers temporaires
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
if os.path.exists(csv_dir):
    shutil.rmtree(csv_dir)
if os.path.exists("spark-warehouse"):
    shutil.rmtree("spark-warehouse")
print("Fichiers temporaires supprimés")
